<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/good_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

기간 입력

거래대금 하위 10%~50%

등락률 상위 50%~10%

클라스로 만들어서 백테스팅 가능하게 ㄱ

In [41]:
!pip install exchange_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [44]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [87]:
class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99]):
      self.period = period
      self.increased = increased
      self.traded = traded

      self.price_dic = {}

      self.start, self.today = self.get_date()
      self.df = self.get_stock_df()
      self.filtered_df = self.get_filtered_df()
      self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_result_df(self):
    for i in self.filtered_df['종목코드']:
      self.price_dic[i] = fdr.DataReader(str(i), self.start, self.today)

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    return self.filtered_df.sort_values('Sharpe', ascending=False)

  def plot_df(self, start_from=0, to=5):
    fig = go.Figure()
    for i in self.price_dic:
      if (i in self.result_df['종목코드'][start_from:to].values):
        fig.add_trace(go.Scatter(x= self.price_dic[i].index, \
                                 y= self.price_dic[i]['Close']/self.price_dic[i]['Close'][0], \
                                 name= self.filtered_df[self.filtered_df['종목코드']==i]['종목명'].item()))

    fig.update_layout(title=self.today,
                      xaxis_title='Date',
                      yaxis_title='Return (%)')
    fig.show()

  def plot_stock(self, id):
    t = stocklist.price_dic[id]
    candlesticks = go.Candlestick(
        x=t.index,
        open=t['Open'],
        high=t['High'],
        low=t['Low'],
        close=t['Close'],
        showlegend=False
    )

    volume_bars = go.Bar(
        x=t.index,
        y=t['Volume'],
        showlegend=False,
        marker={
            "color": "rgba(128,128,128,0.5)",
        }
    )

    fig = go.Figure(candlesticks)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(candlesticks, secondary_y=False
                  )
    fig.add_trace(volume_bars, secondary_y=True)
    fig.update_layout(title= self.filtered_df[self.filtered_df['종목코드']==id]['종목명'].item(), height=800)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        ]
    )
    fig.update_yaxes(title="Price $", secondary_y=True, showgrid=True)
    fig.update_yaxes(title="Volume $", secondary_y=False, showgrid=False)
    fig.show()

In [88]:
stocklist = StockList()

In [89]:
stocklist.result_df

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금,Sharpe,Sortino
2323,022100,포스코 ICT,5750,10970,5220,90.78,386261359,3308635128480,31.656562,73.928925
1273,281740,레이크머티리얼즈,5000,8900,3900,78.00,327080162,2325012750535,28.047417,56.323631
1315,086900,메디톡스,107280,247500,140220,130.70,15168150,2783205215000,26.912193,56.105480
1082,121600,나노신소재,84100,191400,107300,127.59,62532048,7290962240800,25.874926,50.032116
1276,090360,로보스타,20950,37650,16700,79.71,111225886,3726512354750,24.176478,51.847478
1445,000250,삼천당제약,34600,84700,50100,144.80,66780105,3553589067500,22.437620,51.942307
165,002710,TCC스틸,13950,37350,23400,167.74,132849244,2197306180090,22.202717,47.743299
1730,058610,에스피지,16150,31750,15600,96.59,313733024,7642099130370,21.987795,42.516201
744,014580,태경비케이,4395,9210,4815,109.56,312029040,2163418768105,21.943630,41.893796
1197,005160,동국산업,3010,7920,4910,163.12,463912944,2513783673490,20.365089,40.372022


In [90]:
stocklist.plot_df()

In [91]:
stocklist.plot_stock('086900')